In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import scipy
import glob
from tqdm import tqdm
from collections import defaultdict
import eye_metrics_utils
import data_utils
import gaze_entropy


import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, RepeatVector, TimeDistributed
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
csv_files = glob.glob("data/*.csv")

In [3]:
csv_files_one = [v for v in csv_files if "One Gaze-Vergence" in v]
csv_files_two = [v for v in csv_files if "Two Gaze-Vergence" in v]
csv_files_three = [v for v in csv_files if "Three Go-Around Gaze-Vergence" in v]

In [24]:
df_par = pd.read_csv("participant.csv")
par_id_arr = [v[-3:] for v in df_par['ID'].tolist()]
flight_exp_arr = df_par['Flight_hour'].tolist()

group_arr = df_par['Group'].tolist()

for i,e,g in zip(par_id_arr, flight_exp_arr, group_arr):
    print(i, e, g)

032 10 1A
027 18 1A
031 26 1A
028 28 1A
004 35 1A
008 35 1A
010 35 1A
029 40 1A
003 42 1A
007 49 1A
023 50 1A
021 63 2B
006 90 2B
019 98 2B
022 100 2B
015 133 2B
016 200 2B
014 155 2C
005 180 2C
025 200 2C
002 220 2C
001 230 2C
020 230 2C
011 300 2C
017 420 2C
013 23 1A
024 28 1A
018 116 2B
026 150 2C
012 175 2C
009 220 2C
033 1300 2B


In [89]:
X = []
y = []

def norm(df_x):
    train_stats = df_x.describe().transpose()
    return (df_x - train_stats['mean']) / train_stats['std']

for csv_files in [csv_files_two]:
#     part = defaultdict(dict)
    for csv in csv_files:
        par_id = csv[14:17]
        
        if par_id not in par_id_arr:
            continue
#         ret = defaultdict(list)
        df_data = pd.read_csv(csv)
        print(csv, len(df_data))
        exp = flight_exp_arr[par_id_arr.index(par_id)]
        group = group_arr[par_id_arr.index(par_id)]
        for df_slice in data_utils.data_slicing(df_data, window_length = 1200, stride = 900, min_length=1200):
            if (data_utils.check_percentage_null(df_slice) < 0.3): # Ignore data with missing value > 30%
                continue
                
            df_slice.fillna(0.0, inplace=True)
            v = df_slice[['X Pos', 'Y Pos', 'Pupil Diameter']]
            v = norm(v)
            
            X.append(v.values)
#             y.append(exp)
            if "1" in group:
                y.append(0)
            elif "2" in group:
                y.append(1)
            

data\PISSS_ID_001_Approach Two Gaze-Vergence.csv 9554
data\PISSS_ID_002_Approach Two Gaze-Vergence.csv 9430
data\PISSS_ID_003_Approach Two Gaze-Vergence.csv 9368
data\PISSS_ID_004_Approach Two Gaze-Vergence.csv 9862
data\PISSS_ID_005_Approach Two Gaze-Vergence.csv 9245
data\PISSS_ID_006_Approach Two Gaze-Vergence.csv 9739
data\PISSS_ID_007_Approach Two Gaze-Vergence.csv 9677
data\PISSS_ID_008_Approach Two Gaze-Vergence.csv 9923
data\PISSS_ID_009_Approach Two Gaze-Vergence.csv 9243
data\PISSS_ID_010_Approach Two Gaze-Vergence.csv 9923
data\PISSS_ID_011_Approach Two Gaze-Vergence.csv 9492
data\PISSS_ID_012_Approach Two Gaze-Vergence.csv 9431
data\PISSS_ID_013_Approach Two Gaze-Vergence.csv 8691
data\PISSS_ID_014_Approach Two Gaze-Vergence.csv 9307
data\PISSS_ID_015_Approach Two Gaze-Vergence.csv 8812
data\PISSS_ID_016_Approach Two Gaze-Vergence.csv 8259
data\PISSS_ID_017_Approach Two Gaze-Vergence.csv 9184
data\PISSS_ID_018_Approach Two Gaze-Vergence.csv 8937
data\PISSS_ID_019_Approach T

In [90]:
print("Training set dimension:" ,np.array(X).shape)


Training set dimension: (293, 1200, 3)


In [97]:
sum(y)

171

In [7]:
def sqrt_loss_function(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true - y_pred)**2))

In [98]:
X = np.array(X)
y = np.array(y)

In [9]:
scaler = StandardScaler()
y_scale = scaler.fit_transform(y.reshape(-1,1))

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [100]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [12]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(900, 3), return_sequences=True, name = "LSTM_1"))
model.add(LSTM(64, name = "LSTM_2"))

model.add(Dense(10, name = "Fully_connected_1"))
model.add(Dense(1, name = "Fully_connected_2"))

model.compile(loss=sqrt_loss_function, optimizer = tf.keras.optimizers.RMSprop(0.001))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_1 (LSTM)                (None, 900, 128)          67584     
_________________________________________________________________
LSTM_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
Fully_connected_1 (Dense)    (None, 10)                650       
_________________________________________________________________
Fully_connected_2 (Dense)    (None, 1)                 11        
Total params: 117,653
Trainable params: 117,653
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(900,3), name="Convolution_1D_1"))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', name="Convolution_1D_2"))
model.add(MaxPooling1D(pool_size=2, name="MaxPooling_1D_1"))
model.add(Flatten(name="Flatten_1"))
model.add(RepeatVector(1, name="Repeat_vector_1"))
model.add(LSTM(128, activation='relu', return_sequences=True, name = "LSTM_1"))
model.add(Dense(10, activation='relu', name = "Fully_connected_1"))
model.add(Dense(1, name = "Fully_connected_2"))

model.compile(loss=sqrt_loss_function, optimizer = tf.keras.optimizers.RMSprop(0.001))
model.summary()

In [31]:
callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-3,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
        verbose=1)
]

In [15]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=1, callbacks = callbacks, verbose=1)

Train on 264 samples, validate on 89 samples
Epoch 1/20
264/264 [==============================] - 184s 698ms/step - loss: 0.6229 - val_loss: 0.7692
Epoch 2/20
264/264 [==============================] - 201s 761ms/step - loss: 0.6200 - val_loss: 0.7453
Epoch 3/20
264/264 [==============================] - 219s 828ms/step - loss: 0.5928 - val_loss: 0.8462
Epoch 4/20
264/264 [==============================] - 218s 826ms/step - loss: 0.5882 - val_loss: 0.7367
Epoch 5/20
264/264 [==============================] - 236s 894ms/step - loss: 0.5842 - val_loss: 0.7313
Epoch 6/20
264/264 [==============================] - 229s 869ms/step - loss: 0.5709 - val_loss: 0.7335
Epoch 7/20
264/264 [==============================] - 174s 657ms/step - loss: 0.5952 - val_loss: 0.7494
Epoch 00007: early stopping


In [16]:
model.evaluate(X_test, y_test)

89/89 [==============================] - 1s 15ms/step


0.7448194408684634

In [17]:
pred = model.predict(X_test)

In [18]:
p = scaler.inverse_transform(pred)

In [19]:
t = scaler.inverse_transform(y_test)

In [20]:
np.sqrt(mean_squared_error(p.reshape(-1),t))

112.37021542937968

In [21]:
for u,v in zip(p,t):
    print(u,v)

[61.130753] [150.]
[50.474403] [63.]
[101.49334] [155.]
[82.87071] [35.]
[68.68233] [155.]
[78.103264] [100.]
[95.23978] [100.]
[85.31478] [116.]
[73.85091] [155.]
[203.2552] [10.]
[107.48546] [150.]
[95.22088] [150.]
[88.10164] [40.]
[53.844643] [300.]
[86.05946] [175.]
[76.96882] [220.]
[94.4511] [100.]
[74.71586] [18.]
[59.89772] [42.]
[79.085785] [200.]
[80.74635] [98.]
[53.358498] [50.]
[86.18749] [63.]
[84.85354] [220.]
[78.805756] [90.]
[81.36602] [63.]
[74.46299] [18.]
[77.06683] [100.]
[80.3759] [42.]
[58.388824] [155.]
[145.64859] [116.]
[52.66275] [18.]
[69.40344] [420.]
[84.73753] [220.]
[49.519913] [220.]
[86.84679] [28.]
[50.270782] [116.]
[105.251595] [100.]
[97.012215] [26.]
[96.22008] [98.]
[89.778984] [26.]
[79.86883] [35.]
[86.602585] [40.]
[77.36087] [35.]
[68.30212] [200.]
[78.28677] [26.]
[89.83441] [28.]
[46.9804] [49.]
[83.907166] [180.]
[97.42669] [40.]
[72.61808] [28.]
[70.14548] [175.]
[90.23709] [220.]
[189.37503] [420.]
[98.94704] [116.]
[85.926254] [10.]
[

In [22]:
len(t)

89

# classification model

In [115]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(1200, 3), return_sequences=True, name = "LSTM_1"))
model.add(LSTM(64, name = "LSTM_2"))

# model.add(Dense(10, name = "Fully_connected_1"))
model.add(Dense(1, activation="sigmoid", name = "Fully_connected_2"))

model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_1 (LSTM)                (None, 1200, 128)         67584     
_________________________________________________________________
LSTM_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
Fully_connected_2 (Dense)    (None, 1)                 65        
Total params: 117,057
Trainable params: 117,057
Non-trainable params: 0
_________________________________________________________________


In [93]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(1200,3), name="Convolution_1D_1"))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', name="Convolution_1D_2"))
model.add(MaxPooling1D(pool_size=2, name="MaxPooling_1D_1"))
model.add(Flatten(name="Flatten_1"))
model.add(RepeatVector(1, name="Repeat_vector_1"))
model.add(LSTM(128, activation='relu', name = "LSTM_1"))
model.add(Dense(10, activation='relu', name = "Fully_connected_1"))
model.add(Dense(1, activation='sigmoid', name = "Fully_connected_2"))

model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convolution_1D_1 (Conv1D)    (None, 1198, 128)         1280      
_________________________________________________________________
Convolution_1D_2 (Conv1D)    (None, 1196, 64)          24640     
_________________________________________________________________
MaxPooling_1D_1 (MaxPooling1 (None, 598, 64)           0         
_________________________________________________________________
Flatten_1 (Flatten)          (None, 38272)             0         
_________________________________________________________________
Repeat_vector_1 (RepeatVecto (None, 1, 38272)          0         
_________________________________________________________________
LSTM_1 (LSTM)                (None, 128)               19661312  
_________________________________________________________________
Fully_connected_1 (Dense)    (None, 10)              

In [116]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=8, callbacks = callbacks, verbose=1)

Train on 175 samples, validate on 59 samples
Epoch 1/20
175/175 [==============================] - 37s 213ms/step - loss: 0.6777 - accuracy: 0.5886 - val_loss: 0.7162 - val_accuracy: 0.5085
Epoch 2/20
175/175 [==============================] - 37s 212ms/step - loss: 0.6506 - accuracy: 0.6229 - val_loss: 0.7211 - val_accuracy: 0.4915
Epoch 3/20
175/175 [==============================] - 38s 219ms/step - loss: 0.6470 - accuracy: 0.6400 - val_loss: 0.7278 - val_accuracy: 0.5085
Epoch 00003: early stopping


In [122]:
prob = model.predict(X_train)

In [123]:
thresh = 0.6

pred = []
for v in prob:
    if v > thresh:
        pred.append(1)
    else:
        pred.append(0)

In [119]:
# prob

In [120]:
for u,v,p in zip(pred,y_test,prob):
    print(u,v,p)

1 1 [0.621034]
1 1 [0.7384903]
0 1 [0.58146125]
0 1 [0.5038871]
1 0 [0.7363177]
0 0 [0.58066565]
0 0 [0.5799566]
0 1 [0.58131284]
1 0 [0.87280875]
1 0 [0.66131806]
1 0 [0.6499718]
1 0 [0.66942483]
0 1 [0.58417046]
1 1 [0.809629]
0 1 [0.52126116]
1 0 [0.6027039]
1 1 [0.73406214]
1 1 [0.6811725]
1 1 [0.60747516]
1 1 [0.78821087]
1 1 [0.6938323]
0 0 [0.54722005]
1 0 [0.6938436]
1 1 [0.607084]
1 0 [0.737562]
1 0 [0.8020246]
1 1 [0.6811895]
1 1 [0.6436151]
1 1 [0.6456598]
1 0 [0.62175244]
1 0 [0.67832315]
1 1 [0.62808204]
0 0 [0.53077376]
0 1 [0.5853143]
1 0 [0.62011635]
0 1 [0.5894805]
1 1 [0.69877696]
1 0 [0.703298]
0 0 [0.5426033]
1 0 [0.66014796]
1 1 [0.70668316]
1 1 [0.84573853]
0 0 [0.56580323]
1 0 [0.6592716]
1 0 [0.8308723]
1 0 [0.6108274]
1 1 [0.67751896]
1 1 [0.7404653]
0 0 [0.50671816]
0 1 [0.5907571]
1 1 [0.6215226]
1 0 [0.63665813]
1 0 [0.6228131]
0 0 [0.59186006]
0 0 [0.57529914]
0 0 [0.56175154]
0 1 [0.51340914]
1 0 [0.72043115]
1 0 [0.6433616]


In [121]:
from sklearn.metrics import classification_report
target_names = ['group 1', 'group 2']
print(classification_report(y_test, pred, target_names=target_names))

              precision    recall  f1-score   support

     group 1       0.53      0.32      0.40        31
     group 2       0.47      0.68      0.56        28

    accuracy                           0.49        59
   macro avg       0.50      0.50      0.48        59
weighted avg       0.50      0.49      0.48        59

